In [1]:
import os
os.chdir("../")

from wbe_odm import odm
from wbe_odm.odm_mappers import excel_template_mapper

import pandas as pd
import plotly.express as px
import json

In [2]:
data_path = "/Users/jeandavidt/Downloads/Ville de Quebec - All data - v1.1.xlsx"
mapper = excel_template_mapper.ExcelTemplateMapper()
mapper.read(data_path, sheet_names=None)
odm_data = odm.Odm()
odm_data.load_from(mapper)

In [3]:
polygons = odm_data.polygon
polygons['color'] = "grey"
colors = px.colors.qualitative.Plotly
for i in range(len(polygons)):
    polygons.loc[i,"color"] = colors[i%len(colors)]
for col in ["pop", "link"]:
    if col in polygons.columns:
        polygons.drop(columns=[col], inplace=True)
polys = odm_data.get_geoJSON()
with open("polygons.geojson", "w") as f:
    f.write(json.dumps(polys))

In [6]:
def get_site_geoJSON(odm_data):
    def make_point_feature(row):
        props_to_add = ["siteID", "name", "description", "clean_type", "polygonID"]
        return {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["geoLong"], row["geoLat"]],
                },
            "properties": {
                k: row[k] for k in props_to_add
            }
        }
    sites = odm_data.site
    site_types = {
        "wwtpmuc": "Station de traitement des eaux usées municipale pour égouts combinés",
        "pstat": "Station de pompage",
        "ltcf": "Établissement de soins de longue durée",
        "airpln":"Avion",
        "corFcil": "Prison",
        "school": "École",
        "hosptl": "Hôpital",
        "shelter": "Refuge",
        "swgTrck": "Camion de vidange",
        "uCampus": "Campus universitaire",
        "mSwrPpl": "Collecteur d'égouts",
        "holdTnk": "Bassin de stockage",
        "retPond": "Bassin de rétention",
        "wwtpMuS": "Station de traitement des eaux usées municipales pour égouts sanitaires seulement",
        "wwtpInd": "Station de traitement des eaux usées industrielle",
        "lagoon": "Système de lagunage pour traitement des eaux usées",
        "septTnk": "Fosse septique.",
        "river": "Rivière",
        "lake": "Lac",
        "estuary": "Estuaire",
        "sea": "Mer",
        "ocean": "Océan",
    }
    sites = odm_data.site
    sites["clean_type"] = sites["type"].str.lower().map(site_types)
    sites["clean_type"]
    sites["features"] = sites.apply(lambda row: make_point_feature(row), axis=1)
    point_list = list(sites["features"])
    js = {
        "type": "FeatureCollection",
        "features": point_list
    }
    return js

dico_points = get_site_geoJSON(odm_data)
with open("sites.geojson", "w") as f:
    f.write(json.dumps(dico_points))